In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

In [2]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos_titulos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [3]:
len(avisos)

25288

In [2]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [3]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [4]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [5]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [ ]:
vistas.to_csv("vistas_amp.csv")

In [ ]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

In [9]:
no_postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv")

In [10]:
no_postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112366050,1Qdlz6L,0
1,1112112513,0zk5b9a,0
2,1112440465,55ZxxX,0
3,1112311267,mzbNQRY,0
4,1112468890,pzNM0zM,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [ ]:
postulaciones=postulaciones.append(no_postulaciones, ignore_index=True)
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
postulaciones.head()

In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')

In [14]:
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')

In [15]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,pide_administrativo,pide_tecnico,pide_analista,pide_comercial,pide_asistente,pide_ejecutivo,pide_ingeniero,pide_operario,pide_desarollador,cantidad
7400184,1112316469,vVeXmBo,0,FEM,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
3403930,1112285529,3N2OO9a,1,FEM,21.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
10993003,1112243772,625xaM,0,FEM,42.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
2512751,1112204681,dYz1oba,1,FEM,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
3827915,1112364686,ak4kPj0,0,FEM,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
9496291,1111285386,EzeM4p6,1,FEM,25.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
498827,1111067261,2zP6alV,1,FEM,18.0,-,-,-,-,-,...,True,False,False,False,False,False,False,False,False,0.007576
6252047,1112357658,zvxZew6,1,FEM,27.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,False,0.083333
6885257,1112296914,kPjblWZ,1,MASC,39.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,False,False,0.025000
3393763,1112291358,96zjAZZ,1,FEM,38.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,False,False,NaN


In [16]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)

In [17]:
postulaciones.columns

Index(['idaviso', 'idpostulante', 'sepostulo', 'sexo', 'edad', 'Doctorado',
       'Master', 'Otro', 'Posgrado', 'Secundario', 'Terciario/Técnico',
       'Universitario', 'Unnamed: 0', 'titulo', 'nombre_zona',
       'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
       'denominacion_empresa', 'titulo_uppercase', 'pide_hombre', 'pide_mujer',
       'ingles', 'experiencia', 'paquete_office', 'liderazgo', 'secundario',
       'viajar', 'empresa_importante', 'capacitacion',
       'remuneracion_pretendida', 'graduados', 'estudiantes',
       'conocimientos_especificos', 'buen_ambiente', 'discriminacion',
       'pide_vendedor', 'pide_administrativo', 'pide_tecnico', 'pide_analista',
       'pide_comercial', 'pide_asistente', 'pide_ejecutivo', 'pide_ingeniero',
       'pide_operario', 'pide_desarollador', 'cantidad'],
      dtype='object')

In [18]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))

Train:  12148211 Test:  639380


In [19]:
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

In [20]:
x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

In [21]:
vistas = 0
postulaciones = 0
postulantes = 0
vistas2 = 0
avisos = 0
gc.collect()

248

In [ ]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=3000, max_depth=18,n_estimators=50,colsample_bytree=0.9,n_jobs=-1,random_state=0,silent=False,subsample=1,sumsample_freq=1, two_round=True)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=1)

#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/LGBMVUri-1.pkl")

In [ ]:
score=c.score(x_test,y_test)*100
print(score)

In [23]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMVUri-2.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMVUri-1.pkl']

In [24]:
len(avisos)

25288

In [25]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idpostulante','idaviso'],how='left')
print(len(prediccion))
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,pide_administrativo,pide_tecnico,pide_analista,pide_comercial,pide_asistente,pide_ejecutivo,pide_ingeniero,pide_operario,pide_desarollador,cantidad
34620,71297,1112462207,3NzX0vx,MASC,28.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,0.007692
75479,37473,1112413438,3NkG44r,MASC,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.009259
80405,57578,1112451518,ZPdm0E,FEM,33.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
46445,21944,1112280950,6rQM0Jx,MASC,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
96778,96442,1112494071,aXE2Zm,MASC,37.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
6174,84535,1112466576,owadqG3,MASC,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.005263
16405,69365,1112461190,EmMMJJ,MASC,32.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,0.003115
81984,29853,1112374543,Ebepxo,FEM,38.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
85711,34989,1112409694,akVMq,MASC,43.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
50261,20458,1112256432,Bm5Eo90,MASC,19.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,True,False,0.006173


In [25]:
prediccion.dtypes

id                              int64
idaviso_x                       int64
idpostulante                   object
sexo                         category
edad                          float64
Doctorado                    category
Master                       category
Otro                         category
Posgrado                     category
Secundario                   category
Terciario/Técnico            category
Universitario                category
Unnamed: 0                    float64
titulo                         object
nombre_zona                  category
tipo_de_trabajo              category
nivel_laboral                category
nombre_area                  category
denominacion_empresa           object
titulo_uppercase               object
pide_hombre                    object
pide_mujer                     object
ingles                         object
experiencia                    object
paquete_office                 object
liderazgo                      object
secundario  

In [26]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.84778859, 0.15221141],
       [0.67874532, 0.32125468],
       [0.71332192, 0.28667808],
       ...,
       [0.8190345 , 0.1809655 ],
       [0.68409948, 0.31590052],
       [0.95443007, 0.04556993]])

In [27]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [28]:
prediccion['sepostulo']=y_final2

In [30]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,pide_tecnico,pide_analista,pide_comercial,pide_asistente,pide_ejecutivo,pide_ingeniero,pide_operario,pide_desarollador,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,True,True,False,False,False,False,0.0,0.152211
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,True,True,False,False,False,False,0.0,0.321255
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,True,True,False,False,False,False,0.0,0.286678
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.036828
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.046581


In [31]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.152211
1,1,0.321255
2,2,0.286678
3,3,0.036828
4,4,0.046581


In [32]:
prediccion.sepostulo.mean()

0.5262728064696082

In [33]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion100.csv",index=False)